In [1]:
# Donchian Channel Strategy

### Understanding the Donchian Channel Strategy
As mentioned in the previous section of this article, the Donchian Channel Strategy is a fairly simple technical trading strategy that utilizes a visual indicator called Donchian Channels, named after its creator - Richard Donchian, to identify potential breakouts and trend reversals in asset prices.

The Donchian Channels consist of three lines:

- Upper Band — It is the highest price of an asset over a specified period.
- Lower Band — It is the lowest price of an asset over a specified period.
- Middle Band — It is the average of the lower and the upper band.


We will generate a buy signal when the close price or the day’s low is equal to the asset’s lowest price in the past 20 days. Conversely, we will generate a sell signal when the close price or the day’s high is equal to the asset’s highest price in the past 20 days.

In [2]:
!pip install -q yfinance pandas_ta


In [3]:
import yfinance as yf
import numpy as np
import pandas_ta as ta
import numpy as np
import pandas as pd

In [4]:
data = yf.download(f'hcltech.ns', period='10y', progress=False)
data[['low', 'mid', 'high']] = data.ta.donchian(lower_length=20, upper_length=20)

In [5]:
data

,Open,High,Low,Close,Adj Close,Volume,low,mid,high
Date,,,,,,,,,
2014-01-27,352.500000,357.387512,349.149994,356.125000,278.004730,6256364,NaN,NaN,NaN
2014-01-28,354.750000,356.512512,349.524994,353.112488,275.653015,3914664,NaN,NaN,NaN
2014-01-29,352.750000,358.750000,351.524994,355.575012,277.575287,2616276,NaN,NaN,NaN
2014-01-30,355.725006,361.250000,351.950012,358.549988,279.897827,5446576,NaN,NaN,NaN
2014-01-31,359.024994,367.475006,357.524994,365.549988,285.362152,3797684,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...
2024-01-19,1570.000000,1590.000000,1558.050049,1567.949951,1567.949951,2374099,1417.150024,1518.375,1619.599976
2024-01-22,1567.949951,1567.949951,1567.949951,1567.949951,1567.949951,0,1417.150024,1518.375,1619.599976
2024-01-23,1558.900024,1567.900024,1517.050049,1523.650024,1523.650024,3196630,1417.150024,1518.375,1619.599976


```if the day’s low or the close price of the stock is equal to the lower band of the Donchian channel and assign it to a column in the data frame. When this condition is true, we have to initiate a trade and buy the stock the very next day. This will serve as the buy signal.```

In [7]:
## LONG
data['long'] = ((data['Close']==data['low'])|(data['Low']==data['low'])).astype('int')


```if the day’s high or the close price of the stock is equal to the upper band of the Donchian channel and assign it to a column in the data frame. When this condition is true, we should close any open trade the very next day. This will serve as the sell signal.```

In [8]:
## SHORT
data['short'] = ((data['Close']==data['high'])|(data['High']==data['high'])).astype('int')


In [12]:
def trade_donchian(row):
    global trades, trade_open
    row = row.to_dict()

    if((trade_open==True) and (row['long'] == 1)): pass

    elif((trade_open==False) and (row['short'] == 1)): pass

    elif((trade_open==False) and (row['long'] == 1)):
        # open trade
        trade_open = True
        _trade = {
            'buy_date': row['next_date'],
            'buy_price': round(row['next_day_open_price']*1.005,2),
            'sell_price': None,
            'sell_date': None,
        }
        trades.append(_trade)
        del _trade

    elif((trade_open==True) and (row['short'] == 1)):
        # close trade
        trade_open = False
        _trade = trades[-1]
        _trade['sell_date'] = row['next_date']
        _trade['sell_price'] = round(row['next_day_open_price']*0.995,2)
        trades[-1] = _trade
        del _trade



def backtest(stock, period, low=20, high=20):
    global trades, trade_open
    
    # get the stock prices
    data = yf.download(f'{stock}.ns', period=period, progress=False)
    
    data = data.reset_index()
    
    # calculate donchian channels
    data[['low', 'mid', 'high']] = data.ta.donchian(lower_length=low, upper_length=high)
    
    # implement the trading strategy
    data['long'] = ((data['Close']==data['low'])|(data['Low']==data['low'])).astype('int')
    data['short'] = ((data['Close']==data['high'])|(data['High']==data['high'])).astype('int')
     
    # get the next day open price and date
    data['next_day_open_price'] = data['Open'].shift(-1)
    data['next_date'] = data['Date'].shift(-1).astype('string')
    
    trade_open = False
    trades = []
    data.dropna(inplace=True)
    
    cols = ['Date', 'Open', 'Close', 'Adj Close', 'Low', 'High', 
            'low', 'mid', 'high', 'long', 'short', 'next_day_open_price', 
            'next_date']

    data = data[cols]

    data.apply(trade_donchian, axis=1)

    if(len(trades)==0): return None

    x = pd.DataFrame(trades)
    
    # calculate the returns and holding period
    x['buy_date'] = pd.to_datetime(x['buy_date'], format="%Y-%m-%d", dayfirst=True)
    x['sell_date'] = pd.to_datetime(x['sell_date'], format="%Y-%m-%d", dayfirst=True)
    x['returns'] = round(100*(x['sell_price']-x['buy_price'])/x['buy_price'],2)
    x['holding_period'] = (x['sell_date'] - x['buy_date']).dt.days
    x['stock'] = stock
    return x



TRADES = pd.DataFrame()
trades = []
trade_open = False

nifty_50_stocks = ['EICHERMOT','HEROMOTOCO','NESTLEIND','ONGC',
                   'BAJAJ-AUTO','TATASTEEL','GRASIM',
                   'BRITANNIA','BAJFINANCE','M&M','divislab',
                   'HINDUNILVR','HDFCBANK','HDFCLIFE','BHARTIARTL','TCS',
                   'LT','DRREDDY','ULTRACEMCO','SUNPHARMA','NTPC',
                   'TATAMOTORS','UPL','SBIN','HINDALCO','ITC','JSWSTEEL',
                   'COALINDIA','RELIANCE','BPCL','LTIM','MARUTI','HCLTECH',
                   'POWERGRID','WIPRO','SBILIFE','AXISBANK',
                   'ADANIPORTS','ICICIBANK','TITAN','BAJAJFINSV','KOTAKBANK',
                   'TATACONSUM','APOLLOHOSP','INFY','ASIANPAINT',
                   'ADANIENT','INDUSINDBK','TECHM','CIPLA']


for stock in nifty_50_stocks:
    _tr = backtest(stock, '10y', 20, 20)
    if(len(TRADES)==0): TRADES = _tr
    else: TRADES = pd.concat([TRADES, _tr], ignore_index=True)
        

In [13]:
ref: https://python.plainenglish.io/generating-swing-trading-signals-using-donchian-strategy-in-python-7aff3c9ce0a8

NameError: name 'pos_neg' is not defined

## Creating a Momentum Trading Scanner with Dynamic Time Warping
https://freedium.cfd/https://medium.datadriveninvestor.com/creating-a-momentum-trading-scanner-with-dynamic-time-warping-2a4e7ceb1e1c

In [19]:
!pip install numba

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.6/3.6 MB 1.0 MB/s eta 0:00:00m eta 0:00:010:00:010m
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 43.6/43.6 MB 1.1 MB/s eta 0:00:00m eta 0:00:010:00:010m


In [21]:
import time
import numba as nb
import numpy as np
import pandas as pd

from typing import Tuple

import plotly.io as pio
import plotly.graph_objects as go
pio.renderers.default = 'browser'

# Breakout examples in the format [ticker, start date, end date]
BREAKOUTS = [
    ['MNKD', '2020-11-03', '2020-12-10'],
    ['MNKD', '2013-02-27', '2013-03-28'],
    ['LPI', '2021-05-11', '2021-06-18'],
    ['AMD', '2021-07-15', '2021-10-12'],
    ['AMD', '2019-10-03', '2019-12-11'],
    ['AMD', '2018-04-30', '2018-07-25'],
    ['NVAX', '2020-03-12', '2020-05-08'],
    ['LEU', '2021-08-25', '2021-10-08'],
    ['LEU', '2020-11-18', '2020-12-11'],
    ['LEU', '2020-05-06', '2020-05-29'],
    ['FUTU', '2020-04-29', '2020-06-12'],
    ['LAC', '2021-08-18', '2021-10-11'],
    ['LAC', '2020-07-16', '2020-09-11'],
]

# This says we are going to compare a time-series of length LENGTH to all the
# breakout examples (which could be longer or shorter)
LENGTH = 35

# Upper DTW cost threshold to be considered as a breakout candidate
THRESHOLD = 12.23

# Upper and lower dates limits for the plot
PLOT_LOWER_DATE = '2019-10-01'
PLOT_UPPER_DATE = '2020-10-01'


@nb.jit(nopython = True)
def get_cost_matrix(ts1: np.array, ts2: np.array) -> np.array:
    '''
    Get the dynamic time warping cost matrix, which is used to determine
    the warping path and hence the overall cost of the path.
    
    Parameters
    ----------
    ts1 : np.array
        The first time series to compare.
    ts2 : np.array
        The second time series to compare.
    
    Returns
    -------
    C : np.array
        The dynamic time warping cost matrix.
    '''
    
    # Initialise a full cost matrix, filled with np.inf. This is so we can
    # start the algorithm and not get stuck on the boundary
    C = np.full(
        shape = (ts1.shape[0] + 1, ts2.shape[0] + 1), 
        fill_value = np.inf,
    )
    
    # Place the corner to zero, so that we don't have the minimum of 3 infs
    C[0, 0] = 0
    
    for i in range(1, ts1.shape[0] + 1):
        for j in range(1, ts2.shape[0] + 1):
            
            # Euclidian distance between the two points
            dist = abs(ts1[i-1] - ts2[j-1])
            
            # Find the cheapest cost of all three neighbours
            prev_min = min(C[i-1, j], C[i, j-1], C[i-1, j-1])
            
            # Populate the entry in the cost matrix
            C[i, j] = dist + prev_min
            
    return C[1:, 1:]


@nb.jit(nopython = True)
def get_path_cost(C: np.array) -> Tuple[list, float]:
    '''
    Get the optimal path and overall cost of the path.
    
    Parameters
    ----------
    C : np.array
        The DTW cost matrix.
    
    Returns
    -------
    path, cost : Tuple[list, float]
        The optimal path coordinates and the overall cost.
    '''
    
    i = C.shape[0] - 1
    j = C.shape[1] - 1
    
    path = [[i, j]]

    while (i > 0) | (j > 0):
        
        min_cost = min(C[i-1, j-1], C[i-1, j], C[i, j-1])
        
        if min_cost == C[i-1, j-1]:
            i -= 1
            j -= 1
        elif min_cost == C[i-1, j]:
            i -= 1
        elif min_cost == C[i, j-1]:
            j -= 1
        
        path.append([i, j])
        
    return path, C[-1, -1]


@nb.jit(nopython = True)
def standard_scale(ts: np.array) -> np.array:
    return (ts - np.mean(ts))/np.std(ts)


def get_time_series(df: pd.DataFrame,
                    date_start: str,
                    date_end: str) -> np.array:
    '''
    Filter the price dataframe to the specified range, and scale using a z
    score scaling approach.

    Parameters
    ----------
    df : pd.DataFrame
        The price dataframe.
    date_start : str
        Starting date for the time series in the format yyyy-mm-dd
    date_end : str
        Ending date for the time series in the format yyyy-mm-dd

    Returns
    -------
    np.array
        The scaled time series
    '''
    
    df = df[
        (df['Date'] >= date_start)
        & (df['Date'] <= date_end)        
    ]
    
    return standard_scale(df['Close'].values)


@nb.jit(nopython = True)
def get_avg_cost(ts: np.array, breakouts: list) -> float:
    '''
    Compare the time series with all the breakout examples, and return the
    mean of all costs.

    Parameters
    ----------
    ts : np.array
        The time series we are comparing.
    breakouts : list
        A list of time series with the breakout examples.

    Returns
    -------
    float
        The mean of all costs from the time series comparisons.
    '''
    
    costs = []
    
    for i in range(len(breakouts)):
        C = get_cost_matrix(ts, breakouts[i])
        _, path_cost = get_path_cost(C.astype(np.float64))
        
        costs.append(path_cost)
            
    return np.mean(np.array(costs))


def load_breakout_examples() -> list:
    '''
    Load all breakout examples for the time-series comparisons

    Returns
    -------
    list
        A list of scaled time series for comaprisons
    '''
    
    breakouts = []
    
    for b in BREAKOUTS:
         
        df = pd.read_csv(f'data/{b[0]}.csv')
        breakouts.append(get_time_series(df, b[1], b[2]))
    
    return breakouts


@nb.jit(nopython = True)
def run_scanner(close: np.array, 
                breakouts: list,
                length: int,
                threshold: float) -> np.array:
    '''
    Run the scanner over the entire of the stock history, and return an array
    to indicate whether the region is a breakout candidate (1) or not (0)

    Parameters
    ----------
    close : np.array
        The stock closing prices.
    breakouts : list
        A list of time series with the breakout examples.
    length : int
        The lookback period for the scanner.
    threshold : float
        The scanner threshold (values less than this are considered a breakout) 

    Returns
    -------
    np.array
        A binary array indicating the positions where the scanner returns a
        positive result.
    '''
    
    candidates = []
    for idx in range(length, close.shape[0]):
        
        ts = standard_scale(close[idx-length:idx])
        
        cost = get_avg_cost(ts, breakouts)
        
        if cost < threshold:
            candidates.append(1)
        else:
            candidates.append(0)
            
    return np.array(candidates)


def plot_result(df: pd.DataFrame):
    
    df = df[
        (df['Date'] >= PLOT_LOWER_DATE)
        & (df['Date'] <= PLOT_UPPER_DATE)
    ]
    
    df = df.reset_index(drop = True)
    
    df.loc[:, 'breakout_region'] = np.where(
        df['filtered'],
        df['High'].max(),
        df['Low'].min(),
    )
    
    fig = go.Figure()
    
    fig.add_trace(
        go.Candlestick(
            x = df['Date'],
            open = df['Open'],
            high = df['High'],
            low = df['Low'],
            close = df['Close'],
            showlegend = False,        
        ),
    )
    
    fig.add_trace(
        go.Scatter(
            x = df['Date'], 
            y = df['breakout_region'],
            fill = 'tonexty',
            fillcolor = 'rgba(0, 236, 109, 0.2)',
            mode = 'lines',
            line = {'width': 0, 'shape': 'hvh'},
            showlegend = False,
        ),
    )
    
    fig.update_layout(
        xaxis = {'title': 'Date'},
        yaxis = {'range': [df['Low'].min(), df['High'].max()], 'title': 'Price ($)'},
        title = 'TSLA - Breakout Candidates',
        width = 700,
        height = 700,
    )
    
    fig.update_xaxes(
        rangebreaks = [{'bounds': ['sat', 'mon']}],
        rangeslider_visible = False,
    )
    
    fig.show()
    
    return


if __name__ == '__main__':
    
    df = pd.read_csv('TSLA.csv')
    
    t0 = time.time()
    
    candidates = run_scanner(
        df['Close'].values,
        nb.typed.List(load_breakout_examples()),
        LENGTH,
        THRESHOLD,
    )
    
    df = df[LENGTH:].reset_index(drop = True)
    df.loc[:, 'filtered'] = candidates
    
    print('Number of scans performed:', len(df) - LENGTH)
    print('Time taken:', time.time() - t0)
    
    plot_result(df)

FileNotFoundError: [Errno 2] No such file or directory: 'TSLA.csv'

###  the technicals that I use the most are:

For Trend: 50 and 200 day moving averages
For Momentum: Relative Strength Index
For Swings: MACD


ref - gui - https://freedium.cfd/https://medium.com/quant-factory/increase-your-profits-by-50-with-a-simple-yet-elegant-stock-screener-bf63b71512b5

# best books
https://zodiactrading.medium.com/top-10-must-read-books-for-mastering-technical-analysis-4942ff9a231a

## Try on 29 jan

- https://github.com/je-suis-tm/quant-trading
- https://towardsdatascience.com/building-a-comprehensive-set-of-technical-indicators-in-python-for-quantitative-trading-8d98751b5fb
- http://webcache.googleusercontent.com/search?q=cache:https://medium.datadriveninvestor.com/unlocking-profit-building-a-winning-pair-trading-strategy-in-python-cfc8cc30b98a&strip=0&vwsrc=1&referer=medium-parser
- https://readmedium.com/an-algo-trading-strategy-which-made-8-371-a-python-case-study-58ed12a492dc
- https://readmedium.com/en/https:/medium.com/@redeaddiscolll/quantitative-trading-strategies-algorithmic-trading-712f703ed16b
- https://github.com/neurotrader888/TechnicalAnalysisAutomation/tree/main
- https://github.com/gianlucamalato/machinelearning/tree/master
- https://github.com/LastAncientOne/Stock_Analysis_For_Quant/tree/master/Python_Stock